## Position, Bindelta Encodings

The general idea here is to 

In [2]:
import os, tempfile, subprocess, pickle
import numpy as np
import matplotlib.pyplot as plt
import mdtraj as md
from compressstate_ic import *
from runCompressMD import *

In [3]:
rawdesrestrajectory = pickle.load(open("/home/chipbuster/tmp/jupytermicro/mdtraj-pickled.pkl","rb"))
print("Trajectory has " + str(len(rawdesrestrajectory)) + " frames")

trajectory = rawdesrestrajectory[::50]
del rawdesrestrajectory

allnames = ['dres','1lmb','1prs','1pnj','2a3d']
alltrajs = [trajectory, 
            pickle.load(open('/home/chipbuster/tmp/jupytermicro/1lmb-pickled.pkl','rb'))[::50],
            pickle.load(open('/home/chipbuster/tmp/jupytermicro/1prs-pickled.pkl','rb'))[::50],
            pickle.load(open('/home/chipbuster/tmp/jupytermicro/1pnj-pickled.pkl','rb'))[::50],
            pickle.load(open('/home/chipbuster/tmp/jupytermicro/2a3d-pickled.pkl','rb'))[::50]
           ]
desmond_names = allnames[1:]
desmond_trajs = alltrajs[1:]
traj_1lmb = alltrajs[1]
traj_1prs = alltrajs[2]
traj_1pnj = alltrajs[3]
traj_2a3d = alltrajs[4]

# Default settings from paper
numStates = 11
baseSkip = 2 #100 frames, after accounting for input skips

Trajectory has 1987500 frames


In [4]:
import scipy.stats
from math import ceil, log

def calc_min_data_type(lp2):
    if lp2 < 2**8:
        return np.uint8
    elif lp2 < 2**16:
        return np.uint16
    elif lp2 < 2**32:
        return np.uint32
    elif lp2 < 2**64:
        return np.uint64
    else:
        raise ValueError("Requested more than 2^64 bins--you might be fucked.")
            
def calc_ref_frame(frames):
    """Find the modal position of each angle.

    Given a protein trace, finds a configuration so that every 
    frame of the trace can be expressed by the smallest number
    of deltas from the returned frame. This is the (possible
    non-physical) configuration where every internal angle is in
    its modal position.

    input: a (n_frames, n_angles) np.array containing the input
    output: a (1,n_angles) np.array containing the modal angles
    """

    (modes,mode_counts) = scipy.stats.mode(frames)
    return modes.flatten()

def encode_deltas(ref_frame,frames):
    """Encode a protein tracs as deltas from the reference frame.

    inputs: 
        ref_frame: a (1,n_angles) array containing the modal reference frame
        frames: a (n_frames,n_angles) array containing the trace

    output: the tuple (index_stream,delta_stream)
        index_stream: an np.array of ints containing the index of the change
        delta_stream: an np.array of ints containing the bin change
    """
    indices = []
    deltas = []

    for frame in frames:
        diff = frame - ref_frame
        for i in range(len(frame)):
            if diff[i] != 0:
                indices.append(i)
                deltas.append(diff[i])

    return (indices,deltas)

def decode_deltas(self,ref_frame,frames):
    """Delta decoder. Used to check product of decode"""
    for frame in frames:
        frame += ref_frame

def gen_dzr_triple(data, maxval):
    """Generate the triple associated with the compression for each type
    
    Example: given an array containing indices, generate a frame of all zeros
    the same size as the indices, a random array of the same size, and the 
    original data.
    
    returns: (data, zeros, random)
    """
    
    zeros = np.zeros(np.shape(data), dtype=data.dtype)
    random = np.randint(low=0,high=maxval,size=np.shape(data),dtype=data.dtype)
    return (data, zeros, random)
    
        
class PBDCompressionData():
    def __init__(self,univ,bincount):
        self.bincount = bincount
        
        frames = extraction.convert_IC(univ)
        binned = np.digitize(frames,
                                  np.linspace(-180,180,num=bincount),
                                  right=True)
        self.ref_frame = self.calc_ref_frame(frames)
        (indices,deltas) = self.encode_deltas(self.ref_frame, frames)
        
        global_maxval = max(bincount, len(univ))
        self.dtype = 
        
        self.indices = np.array(indices, dtype=self.idtype)
        self.deltas = np.array(deltas, dtype=self.ddtype)

    def size_data(self):
        isize = self.compressed_data_size(self.indices.tobytes())
        dsize = self.compressed_data_size(self.deltas.tobytes())
        refsize = self.compressed_data_size(self.ref_frame.tobytes())
        return (isize,dsize,refsize)

    def size_zeros(self):
        izsize = self.compressed_data_size(np.zeros(np.shape(self.indices),dtype=self.idtype))
        dzsize = self.compressed_data_size(np.zeros(np.shape(self.deltas),dtype=self.ddtype))
        rzsize = self.compressed_data_size(np.zeros(np.shape(self.ref_frame),dtype=self.rdtype))
        return (izsize, dzsize,rzsize)

    def size_random(self):
        ran_i = np.random.randint(low=0,high=np.max(self.indices),size=np.shape(self.indices),dtype=self.idtype)
        ran_d = np.random.randint(low=0,high=np.max(self.deltas),size=np.shape(self.deltas),dtype=self.ddtype)
        ran_r = np.random.randint(low=0,high=np.max(self.bincount),size=np.shape(self.ref_frame),dtype=self.rdtype)
        irsize = self.compressed_data_size(ran_i)
        drsize = self.compressed_data_size(ran_d)
        rrsize = self.compressed_data_size(ran_r)
        return (irsize, drsize, rrsize)

    def compressed_data_size(self, obj):
        """Get size of compressed data. Input needs to be a bytearray."""
        compressedStateData = lzma.compress(obj, 
                                    format=lzma.FORMAT_RAW,
                                    check=lzma.CHECK_NONE,
                                    preset=None,
                                    filters=lzmaFilters)
        return len(compressedStateData)

    def get_compression_ratios(self):
        """Compute compression ratio of data in system."""
        (C_di,C_dd,C_dr) = self.size_data()
        (C_0i,C_0d,C_0r) = self.size_zeros()
        (C_1i,C_1d,C_1r) = self.size_random()
        
        
        print(np.array([[C_di,C_dd,C_dr],
                       [C_0i,C_0d,C_0r],
                       [C_1i,C_1d,C_1r]]))
        
        C_d = C_di + C_dd + C_dr
        C_0 = C_0i + C_0d + C_0r
        C_1 = C_1i + C_1d + C_1r
        eta = (C_d - C_0) / (C_1 - C_0)

        return eta
    
    def testfunc(self):
        zeros = np.zeros(np.shape(self.ref_frame),dtype=self.rdtype)
        random = np.random.randint(low=0,high=np.max(self.bincount),size=np.shape(self.ref_frame),dtype=self.rdtype)
        
        print(np.shape(zeros))
        print(np.shape(random))
        print(np.shape(self.ref_frame))
        
        print(self.compressed_data_size(zeros.tobytes()))
        print(self.compressed_data_size(random.tobytes()))
        print(self.compressed_data_size(self.ref_frame.tobytes()))
        
